<a href="https://colab.research.google.com/github/AnSaradar/NYC_Taxi_Driver_Analysis/blob/main/NYC_Taxi_Trips_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup & Import Dask**

In [10]:
# External dependencies
import os
import shutil
import warnings
import numpy as np
import zipfile
import matplotlib.pyplot as plt
from os import path


warnings.filterwarnings("ignore")

In [11]:
!pip install dask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
dd.set_option("display.max_rows", None)

# Loading the Data

In [43]:
import dask.dataframe as dd

In [44]:
zip_path = "/content/drive/MyDrive/Datasets/Copy of nyc_taxi_data.zip"
extract_dir = '/content/sample_data'

In [45]:
shutil.unpack_archive(zip_path, extract_dir)


In [46]:
df = dd.read_parquet('/content/sample_data/nyc_taxi_alt/*.parquet')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1.0,1.5,1.0,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1.0,2.6,1.0,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3.0,0.0,1.0,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5.0,0.0,1.0,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5.0,0.0,2.0,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN,NaN


# **Discovering the Data**

In [16]:
from dask import delayed, compute, visualize, dataframe
from dask.distributed import Client

In [ ]:
print("Number of rows:", df.shape[0].compute())
print("Number of columns:", len(df.columns))

Number of rows: 179311700
Number of columns: 19


In [ ]:
df.isnull().sum().compute()

VendorID                         0
tpep_pickup_datetime             0
tpep_dropoff_datetime            0
passenger_count            3872186
trip_distance                    0
RatecodeID                 3872186
store_and_fwd_flag         3872186
PULocationID                     0
DOLocationID                     0
payment_type                     0
fare_amount                      0
extra                            0
mta_tax                          0
tip_amount                       0
tolls_amount                     0
improvement_surcharge            0
total_amount                     0
congestion_surcharge       8660504
airport_fee              115760994
dtype: int64

In [ ]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [ ]:
df.airport_fee.value_counts().compute()

 0.00    59042827
 1.25     4471301
-1.25       36577
 0.50           1
Name: airport_fee, dtype: int64

In [8]:
df.passenger_count.value_counts().compute()

1.0      126123945
2.0       26414914
3.0        7162781
5.0        5554004
0.0        3482482
6.0        3425107
4.0        3274587
7.0            809
8.0            524
9.0            359
96.0             1
112.0            1
Name: passenger_count, dtype: int64

In [32]:
df.congestion_surcharge.value_counts().compute()

 2.50    145092336
 0.00     14554138
-2.50       452796
 0.75         1195
 2.75          673
 0.50           44
 1.00           25
-0.75            9
 1.50            4
 0.80            2
-1.50            1
 0.30            1
 1.80            1
 2.25            1
 3.00            1
Name: congestion_surcharge, dtype: int64

In [ ]:
df.describe().compute()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
count,1.793117e+08,1.754395e+08,1.793117e+08,1.754395e+08,1.793117e+08,1.793117e+08,1.793117e+08,1.793117e+08,1.793117e+08,1.793117e+08,1.793117e+08,1.793117e+08,1.793117e+08,1.793117e+08,1.706512e+08,6.355071e+07
mean,1.671094e+00,1.492676e+00,4.405538e+00,1.143369e+00,1.637911e+02,1.617755e+02,1.349501e+00,1.262162e+01,1.064549e+00,4.952468e-01,3.319861e+00,4.097364e-01,3.020870e-01,1.969599e+01,2.241222e+00,8.722807e-02
std,4.698158e-01,1.119885e+00,4.219229e+02,2.890464e+00,6.599441e+01,7.044308e+01,7.198644e-01,1.050185e+04,3.736028e+01,3.733934e+01,1.050032e+04,1.847462e+00,6.781555e-02,1.986252e+02,7.857634e-01,3.212864e-01
min,1.000000e+00,0.000000e+00,-3.726453e+04,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-1.333914e+08,-6.000000e+01,-5.500000e-01,-4.932200e+02,-9.999000e+01,-1.000000e+00,-2.567800e+03,-2.500000e+00,-1.250000e+00
25%,2.000000e+00,1.000000e+00,1.390000e+00,1.000000e+00,1.370000e+02,1.320000e+02,1.000000e+00,8.500000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,3.000000e-01,1.280000e+01,2.500000e+00,0.000000e+00
50%,2.000000e+00,1.000000e+00,2.530000e+00,1.000000e+00,1.630000e+02,1.630000e+02,1.000000e+00,1.250000e+01,1.000000e+00,5.000000e-01,2.340000e+00,0.000000e+00,3.000000e-01,1.716000e+01,2.500000e+00,0.000000e+00
75%,2.000000e+00,2.000000e+00,6.420000e+00,1.000000e+00,2.360000e+02,2.360000e+02,4.000000e+00,2.610000e+01,2.750000e+00,5.000000e-01,4.820000e+00,0.000000e+00,3.000000e-01,3.444000e+01,2.500000e+00,0.000000e+00
max,2.000000e+00,1.120000e+02,3.896785e+05,9.900000e+01,2.650000e+02,2.650000e+02,5.000000e+00,9.983100e+05,5.000008e+05,5.000005e+05,1.333914e+08,3.288000e+03,1.000000e+00,1.084772e+06,4.500000e+00,1.250000e+00


In [33]:
df.payment_type.value_counts().compute()

1    122751631
2     40466949
3       758470
4       545424
5           17
Name: payment_type, dtype: int64

# Cleaning the Missing & Outlier Values 

## Passanger Count

In [47]:
client = Client()

df = df.map_partitions(lambda partition: partition.loc[partition['passenger_count'] <= 4])

client.close()


INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:35509
INFO:distributed.scheduler:  dashboard at:           127.0.0.1:42651
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:33079'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:37991'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:46517', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:46517
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:36156
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:35037', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:35037
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:36158
INFO:distributed.scheduler:Receive client connection: Client-4dcfd226-06dd-11ee-8e08-0242ac1c000c
INF

## Trip Distance

In [48]:
client = Client()

df = df.map_partitions(lambda partition: partition.loc[partition['trip_distance'] > 0])

client.close()

INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:39733
INFO:distributed.scheduler:  dashboard at:           127.0.0.1:32871
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:45867'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:33919'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:45473', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:45473
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:43122
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:36025', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:36025
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:43120
INFO:distributed.scheduler:Receive client connection: Client-5044d42a-06dd-11ee-8e08-0242ac1c000c
INF

## Airport Fee

In [49]:
df = df.loc[~df['airport_fee'].isin([0.50, -1.25])]
df['airport_fee'] = df['airport_fee'].fillna(0.0)

## RatecodeID & store_and_fwd_flag

In [51]:
df = df.dropna(subset=['RatecodeID', 'store_and_fwd_flag'])

##Congestion Surcharge

In [52]:
df = df.loc[~df['congestion_surcharge'].isin([-2.50,-0.75,-1.50 ])]
df['congestion_surcharge'] = df['congestion_surcharge'].fillna(2.50)

## Fare Amount

In [53]:
client = Client()

df = df.map_partitions(lambda partition: partition.loc[partition['fare_amount'] > 0])

client.close()

INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:40325
INFO:distributed.scheduler:  dashboard at:           127.0.0.1:34247
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:37199'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:45499'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:38671', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:38671
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:38222
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:45353', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:45353
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:38232
INFO:distributed.scheduler:Receive client connection: Client-5c95a26a-06dd-11ee-8e08-0242ac1c000c
INF

## Extra

In [54]:
client = Client()

df = df.map_partitions(lambda partition: partition.loc[partition['extra'] >= 0])

client.close()

INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:43207
INFO:distributed.scheduler:  dashboard at:           127.0.0.1:39123
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:45503'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:45481'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:44007', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:44007
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:57908
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:34455', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:34455
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:57920
INFO:distributed.scheduler:Receive client connection: Client-5e0935a9-06dd-11ee-8e08-0242ac1c000c
INF

##MTA Tax

In [55]:
client = Client()

df = df.map_partitions(lambda partition: partition.loc[partition['mta_tax'] >= 0])

client.close()

INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:40653
INFO:distributed.scheduler:  dashboard at:           127.0.0.1:43307
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:46699'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:40651'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:33817', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:33817
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:51942
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:33785', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:33785
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:51928
INFO:distributed.scheduler:Receive client connection: Client-5f7bdb25-06dd-11ee-8e08-0242ac1c000c
INF

##Improvement Surcharge

In [56]:
client = Client()

df = df.map_partitions(lambda partition: partition.loc[partition['improvement_surcharge'] >= 0])

client.close()

INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:38361
INFO:distributed.scheduler:  dashboard at:           127.0.0.1:43723
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:35909'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:33527'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:42727', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:42727
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:45606
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:39605', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:39605
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:45608
INFO:distributed.scheduler:Receive client connection: Client-60cdb723-06dd-11ee-8e08-0242ac1c000c
INF

## Tip Amount

In [57]:
client = Client()

df = df.map_partitions(lambda partition: partition.loc[partition['tip_amount'] >= 0])

client.close()

INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:40909
INFO:distributed.scheduler:  dashboard at:           127.0.0.1:41309
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:41595'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:44487'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:33533', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:33533
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:47552
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:42941', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:42941
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:47554
INFO:distributed.scheduler:Receive client connection: Client-636984b0-06dd-11ee-8e08-0242ac1c000c
INF

## Tolls Amount

In [58]:
client = Client()

df = df.map_partitions(lambda partition: partition.loc[partition['tolls_amount'] >= 0])

client.close()

INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:42915
INFO:distributed.scheduler:  dashboard at:           127.0.0.1:33845
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:37213'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:37537'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:46707', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:46707
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:35214
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:40085', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:40085
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:35204
INFO:distributed.scheduler:Receive client connection: Client-651b8949-06dd-11ee-8e08-0242ac1c000c
INF

##Total Amount

In [59]:
client = Client()

df = df.map_partitions(lambda partition: partition.loc[partition['total_amount'] > 0])

client.close()

INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:37343
INFO:distributed.scheduler:  dashboard at:           127.0.0.1:39663
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:45081'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:40599'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:42977', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:42977
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:38700
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:41055', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:41055
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:38696
INFO:distributed.scheduler:Receive client connection: Client-66922501-06dd-11ee-8e08-0242ac1c000c
INF

In [61]:
client = Client()

column_missing_counts = df.map_partitions(lambda partition: partition.isnull().sum()).compute()

client.close()

print(column_missing_counts)

INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:35533
INFO:distributed.scheduler:  dashboard at:           127.0.0.1:33009
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:32927'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:44957'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:32769', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:32769
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:59440
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:37247', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:37247
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:59446
INFO:distributed.scheduler:Receive client connection: Client-63dc683f-06de-11ee-8e08-0242ac1c000c
INF

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
                        ..
tolls_amount             0
improvement_surcharge    0
total_amount             0
congestion_surcharge     0
airport_fee              0
Length: 11571, dtype: int64


In [65]:
client = Client()

shape = df.map_partitions(lambda partition: partition.shape).compute()

client.close()

print(shape)

INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:35141
INFO:distributed.scheduler:  dashboard at:           127.0.0.1:36473
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:38917'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:41691'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:44831', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:44831
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:51460
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:36231', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:36231
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:51464
INFO:distributed.scheduler:Receive client connection: Client-fd3cf9a9-06e3-11ee-8e08-0242ac1c000c
INF

0      (279196, 19)
1      (280966, 19)
2      (280570, 19)
3      (280182, 19)
4      (281369, 19)
           ...     
604    (269053, 19)
605    (270169, 19)
606    (268652, 19)
607    (268931, 19)
608    (149303, 19)
Length: 609, dtype: object


In [63]:
client = Client()

df.to_parquet('/content/sample_data/cleaned_data.parquet', engine='pyarrow')

client.close()

INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:41855
INFO:distributed.scheduler:  dashboard at:           127.0.0.1:46271
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:44295'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:35711'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:34649', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:34649
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:60778
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:36463', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:36463
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:60788
INFO:distributed.scheduler:Receive client connection: Client-282ee36f-06e1-11ee-8e08-0242ac1c000c
INF

# Building A Time Series 

In [ ]:
df['pickup_timestamp'] = dd.to_datetime(df['tpep_pickup_datetime'])
df['dropoff_timestamp'] = dd.to_datetime(df['tpep_dropoff_datetime'])
df = df.set_index(['pickup_timestamp'])